# Creating EC, EO indicators for RS recordings

## Import

In [1]:
import os 
os.chdir('/CSNG/studekat/ripple_band_project/code')

In [2]:
from functions_analysis import *
import pandas as pd
import numpy as np
import yaml
import pickle
import neo
import elephant
import quantities as pq

## Parameters

In [3]:
with open("params_analysis.yml") as f:
    params = yaml.safe_load(f)
### AUX = params['aux']

DATA_FOLDER = params['data_folder']
DATES = params['dates']
MONKEY_LIST = ['L','N','F','A']

## Sandbox

In [10]:
name = f'eyes_indic_monkey_{monkey}_RS_date_{date}_common_times'
with open(f'{MAIN_FOLDER}/metadata/EC_EO_indicators/{name}.pkl', 'rb') as f:
    your_data = pickle.load(f)

In [11]:
your_data

{'EC': array([0., 0., 0., ..., 0., 0., 0.], shape=(2460527,)),
 'EO': array([1., 1., 1., ..., 0., 0., 0.], shape=(2460527,))}

## Creating EC, EO indicators, common times only

In [9]:
MAIN_FOLDER = '/CSNG/studekat/ripple_band_project'

for monkey in MONKEY_LIST:
    for date in DATES[monkey]['RS']:
        df_eyes = pd.read_csv(f'{DATA_FOLDER}/macaque{monkey}_RS_{date}/epochs_macaque{monkey}_RS_{date}.csv')
        duration_rec = params['times_all_arr'][monkey]['RS'][date][1] - params['times_all_arr'][monkey]['RS'][date][0] ### duration when all arrays are on
        ### loading indicator for the whole duration of array 1 rec time, sometimes not even that time is covered, a bit shorter
        EC_indic = create_indicator(df_eyes,start_col='t_start',stop_col='t_stop',state_col='state',
                             positive_state='Closed_eyes',mult_factor=1000)
        EO_indic = create_indicator(df_eyes,start_col='t_start',stop_col='t_stop',state_col='state',
                             positive_state='Open_eyes',mult_factor=1000)
        ### cutting only the common time
        cut_EC_indic = np.zeros(duration_rec)
        cut_EC_indic[:EC_indic.shape[0]] = EC_indic[:duration_rec]
        cut_EO_indic = np.zeros(duration_rec)
        cut_EO_indic[:EO_indic.shape[0]] = EO_indic[:duration_rec] 

        ### saving as a dictionary
        eyes_indic_dict = {}
        eyes_indic_dict['EC'] = cut_EC_indic
        eyes_indic_dict['EO'] = cut_EO_indic

        name = f'eyes_indic_monkey_{monkey}_RS_date_{date}_common_times'
        ensure_dir_exists(f'{MAIN_FOLDER}/metadata/EC_EO_indicators/')
        with open(f'{MAIN_FOLDER}/metadata/EC_EO_indicators/{name}.pkl', 'wb') as file:
            pickle.dump(eyes_indic_dict, file)